# Анализ комментариев на токсичность
___

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В моем распоряжении набор данных с разметкой о токсичности правок.

## Признаки
___

Данные находятся в файле `toxic_comments.csv`. 

- **`text`** — текст комментария;
- **`toxic`** — целевой признак.

## Содержание
___

<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-дерева-решений" data-toc-modified-id="Модель-дерева-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель дерева решений</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a>

## Подготовка
___

Импортирую необходимые для анализа библиотеки:

In [1]:
import pandas as pd
import numpy as np
import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import f1_score

import re

Прочитаю файл с данными:

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Датасет имеет 159292 строк и 2 столбца:
- `text` - столбец, содержащий текстовку комментариев;
- `toxic` - целевой столбец, содержащий информацию о том, токсичный комментарий (1) или нет (0).

Проверю, имеются ли пропуски в данных:

In [5]:
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

Для обработки датасета необходимо лемматизировать текст комментариев. Для этого создам соответствующую функцию:

In [6]:
def lemm_text(text):    
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_text

Для обработки датасета необходимо очистить и привести к одному виду текст комментариев. Для этого создам соответсвующую функцию:

In [7]:
def clear_text(text):    
    stop_words = set(nltk_stopwords.words('english'))
    text = text.lower()
    word_list = re.sub(r'[^a-z ]', ' ', text).split()
    word_notstop_list = [w for w in word_list if not w in stop_words]
    return ' '.join(word_notstop_list)

Создам соответсвующие столбцы с обработанными данными:

In [8]:
data['clear_text'] = data['text'].apply(clear_text)

In [9]:
data['lemm_text'] = data['clear_text'].apply(lemm_text)

In [10]:
data.head()

,Unnamed: 0,text,toxic,clear_text,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...,explanation edits made username hardcore metal...
1,1,D'aww! He matches this background colour I'm s...,0,aww matches background colour seemingly stuck ...,aww match background colour seemingly stuck th...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man really trying edit war guy constantly ...,hey man really trying edit war guy constantly ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestions improvement wondered sec...,make real suggestion improvement wondered sect...
4,4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page,sir hero chance remember page


## Обучение
___

Выделю столбец с леммитизированными данными в корпус:

In [11]:
corpus = data['lemm_text']

In [12]:
kfold = KFold(n_splits=5, random_state=12345, shuffle=True)

Посчитаю `TF-IDF` данных:

In [13]:
count_tf_idf = TfidfVectorizer()
tf_idf = count_tf_idf.fit_transform(corpus)

Посмотрим на матрицу:

In [14]:
tf_idf.shape

(159292, 158376)

Выделю обучающую и тестовую части данных:

In [15]:
features_train, features_test, target_train, target_test = train_test_split(
    tf_idf, data['toxic'].values, test_size=0.2, stratify=data['toxic'].values, shuffle=True, random_state=12345)

### Модель логистической регрессии

Обучу модель логистической регрессии и посчитаю `F-1 меру`:

In [16]:
model_lr = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=12345)

In [17]:
print('f1 score =', cross_val_score(model_lr, features_train, target_train, cv=kfold, scoring='f1').mean())

f1 score = 0.7529384802135397


### Модель дерева решений

In [18]:
model_dtc = DecisionTreeClassifier(class_weight='balanced', random_state=12345)

In [19]:
print('f1 score =', cross_val_score(model_dtc, features_train, target_train, cv=kfold, scoring='f1').mean())

f1 score = 0.6400567407112728


## Выводы
___

Мной была проведена обработка исходных данных, проанализированы две различные модели обучения: LogisticRegression и DecisionTreeClassifier. Лучшее значение F-1 меры показала модель LogisticRegression. На ней я и обучил тестовые данные и получил F-1 меру со значением больше 0.75.